<a href="https://colab.research.google.com/github/bansalhim/Deep-learning-assignment/blob/main/Assignment_4_deep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import pandas as pd
data = pd.read_csv("/content/multiple_linear_regression_dataset.csv")  # replace path if needed

print("Shape:", data.shape)
print("Columns:", data.columns.tolist())
display(data.head())# for first 5 rows
print("\nInfo:")# printing all information
display(data.info())
print("\nSummary statistics:")
display(data.describe())


Shape: (20, 3)
Columns: ['age', 'experience', 'income']


,age,experience,income
0,25,1,30450
1,30,3,35670
2,47,2,31580
3,32,5,40130
4,43,10,47830



Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   age         20 non-null     int64
 1   experience  20 non-null     int64
 2   income      20 non-null     int64
dtypes: int64(3)
memory usage: 612.0 bytes


None


Summary statistics:


,age,experience,income
count,20.000000,20.000000,20.000000
mean,39.650000,6.200000,40735.500000
std,10.027725,4.124382,8439.797625
min,23.000000,1.000000,27840.000000
25%,31.500000,3.750000,35452.500000
50%,40.000000,5.000000,40190.000000
75%,47.000000,9.000000,45390.000000
max,58.000000,17.000000,63600.000000


In [ ]:
#Which columns are inputs?
#Ans->The dataset contains 20 samples with two input features 'age' and 'experience'
#Which column is the output?
#ans->Income is output.
#How many features does your model need to handle?
#Ans->The model needs to handle 2 features — age and experience — since these are the two input variables used to predict the output income.

In [ ]:
# Step 2: Separate Inputs and Output

import numpy as np

X = data[["age", "experience"]].values
y = data["income"].values# The model must learn to map X -> y.

print("X shape:", X.shape)
print("y shape:", y.shape)


X shape: (20, 2)
y shape: (20,)


In [ ]:
#What is the shape of X?
#ans->X has shape (20, 2) meaning 20 samples and 2 input features.
#What is the shape of y?
#ans-> y has shape (20,) meaning 20 target values (income for each person).
#Why does X have 2 columns but y only one?
#ans->So that the model will learn a relationship between age, experience, and income.

In [ ]:
# Step 3: Initialize Model Parameters
# Initializes weights and bias with zeros.
# Learning starts from initial guesses so these will be updated to minimize error.

n_features = X.shape[1]
w = np.zeros(n_features)
b = 0.0

print("Initial weights:", w)
print("Initial bias:", b)


Initial weights: [0. 0.]
Initial bias: 0.0


In [ ]:
#Why do we need one weight per feature?
#ans->Each feature (e.g., age and experience) contributes separately to the output.the model learns how strongly each input affects the prediction, so it needs one weight for each feature to control its influence on the final income estimate.
#Why is bias separate?
#asn->The bias allows the regression line to shift up or down independently of the input features.Without it, the model would always pass through the origin (0, 0), which is rarely correct in real-world data.
#Would initializing with large values be risky?
#ans->Yes — large initial weights can cause the predicted outputs and errors to explode, leading to unstable or diverging gradient-descent updates.Starting with small or zero values keeps training stable and allows the model to gradually learn appropriate weights.

In [ ]:
# Step 4: Define the Forward Pass (Prediction)
# Computes predicted income as a linear combination of inputs.
# This is the perceptron’s basic operation without an activation function.

def predict(X, w, b):
    y_hat = X.dot(w) + b
    return y_hat


In [ ]:
#Why is there no activation function?
#ans->Because this is a regression problem, not classification.We want the output to be a continuous numeric value (income), not limited to 0 or 1.An activation function like sigmoid or ReLU would restrict or distort the output range, which isn’t needed for regression.
#What kind of values can y_hat take?
#ans->y_hat can take any real (continuous) value, positive or negative, depending on the input features and weights.In this dataset, since income is always positive, we expect y_hat to fall roughly within the range of actual incomes (around ₹28,000–₹63,000).
#How is this different from logistic regression?
#ans->Logistic regression adds a sigmoid activation to the linear combination of inputs, compressing the output into the range (0, 1) to represent a probability.In contrast, linear regression  has no activation function, allowing unrestricted numeric outputs for predicting continuous values like income.

In [ ]:
# Step 5: Define the Loss Function (Mean Squared Error)
# Calculates average squared difference between predictions and true values.
# The model needs a way to measure how wrong its predictions are.

def mean_squared_error(y, y_hat):
    loss = np.mean((y_hat - y) ** 2)
    return loss


In [ ]:
#Why square the error?
#ans->Squaring the error makes all differences positive, so that overestimations and underestimations don’t cancel each other out.It also emphasizes larger errors more strongly, forcing the model to pay more attention to big mistakes.
#What happens if one prediction is very wrong?
#ans->A single large rror will increase the overal loss a lot, because the squared term grows rapidly for big differences.his sensitivity helps the model focus on reducing those large mistakes during training.
#Why not just take absolute error?
#ans->The absolute error is harder to optimize because it’s not differentiable at zero — gradient descent works best with smooth, differentiable functions like MSE.Also, squaring makes the loss function mathematically easier to handle and ensures a single, stable minimum.

In [ ]:
# Step 6: Compute Gradients
# Computes gradients of MSE loss with respect to weights and bias.
# Gradients tell the model how to change parameters to reduce the error.

def compute_gradients(X, y, y_hat):
    N = len(y)
    dw = (2 / N) * X.T.dot(y_hat - y)
    db = (2 / N) * np.sum(y_hat - y)
    return dw, db


In [ ]:
# Why does X appear in dw but not in db?
# ans-> Each weight corresponds to a feature, so the gradient with respect to each weightdepends on that feature's value (X). Bias is a constant term applied to all samples,so it does not multiply any input feature.

# Why does the error term appear everywhere?
# ams-> The error (y_hat - y) shows how far predictions are from true values.It affects every parameter update because both weights and bias must reduce that error.

# What happens if error is zero?
# ans-> If the error is zero, gradients become zero.That means the model has perfectly fit the data, and no further updates are needed.


In [ ]:
# Step 7: Update Parameters (Gradient Descent)
# Updates weights and bias using learning rate and computed gradients.
# This step moves parameters toward values that reduce loss.

def update_parameters(w, b, dw, db, lr):
    w = w - lr * dw
    b = b - lr * db
    return w, b

# The learning rate (lr) controls the update size.
# If lr is too large → unstable training; too small → slow learning.


In [ ]:
# Step 8: Training Loop
# Repeats prediction, loss calculation, gradient computation, and parameter update.

lr = 0.0001
epochs = 1000

for epoch in range(epochs):
    y_hat = predict(X, w, b)
    loss = mean_squared_error(y, y_hat)
    dw, db = compute_gradients(X, y, y_hat)
    w, b = update_parameters(w, b, dw, db, lr)

    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss:.4f}")

print("\nFinal weights:", w)
print("Final bias:", b)


Epoch 0, Loss: 1727049635.0000
Epoch 100, Loss: 66491868.5531
Epoch 200, Loss: 61752567.2012
Epoch 300, Loss: 58616531.0785
Epoch 400, Loss: 56528801.5395
Epoch 500, Loss: 55126542.0295
Epoch 600, Loss: 54172526.9489
Epoch 700, Loss: 53511656.1429
Epoch 800, Loss: 53042523.7280
Epoch 900, Loss: 52698829.5633

Final weights: [ 764.75405919 1371.03430441]
Final bias: 321.73641174472493


In [ ]:
# Does loss decrease over time?
# -> Yes. The loss decreases gradually from 1.7e9 to around 5.2e7,showing that the model is learning and improving its predictions over epochs.

# What happens if it increases?
# -> If loss increases instead of decreasing, it means the learning rate is too high or the gradients are incorrect. The model would then diverge instead of converging.

# How do learning rate and epochs interact?
# -> The learning rate controls the size of each update step. A smaller learning rate requires more epochs to reach a good minimum, while a larger rate learns faster but risks overshooting and instability.


In [ ]:
# Step 9: Final Evaluation
# Prints final weights and bias, and predicts income for a new sample.
# To test the trained model on new input.

new_sample = np.array([40, 10])   # means : age = 40, experience = 10
predicted_income = new_sample.dot(w) + b
print(f"Predicted income for (age=40, experience=10): {predicted_income:.2f}")


Predicted income for (age=40, experience=10): 44622.24


In [ ]:
# Is the prediction reasonable?
# -> Yes. The predicted income lies within the general range of the dataset (around 28k–63k),which makes sense for the given age and experience values. It shows that the model has learned a realistic linear relationship.

# Does it interpolate smoothly?
# -> Yes. Because the model is linear, chnges in age or experience produce smooth and proportional changes in predicted income — no sudden jumps or drops.

# Why is this better than threshold rules?
# -> Threshold rules (like those in a perceptron) make hard yes/no decisions. In contrast, linear regression provides continuous numeric predictions,allowing smoother interpolation and more precise estimation of outcomes.
